In [120]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [121]:
data = pd.read_csv("data_set_final2.csv", low_memory=False)

In [122]:
data.drop(columns=['rolling_avg_DESCIDA_PRECO','rolling_avg_AUMENTO_PRECO','rolling_avg_SKUS_DOWN','rolling_avg_items','rolling_avg_sales','rolling_avg_SKUS_UP'], inplace=True)

In [123]:
# Ensure 'DATA_VENDA' is parsed as datetime
data['DATA_VENDA'] = pd.to_datetime(data['DATA_VENDA'])
data['day_of_week'] = data['DATA_VENDA'].dt.day_name()

# Create lag and rolling mean features
data['lag_7'] = data.groupby('LOJA')['VALOR_VENDA'].shift(7)
data['rolling_mean_15'] = data.groupby('LOJA')['VALOR_VENDA'].shift(1).rolling(window=15).mean()

# Handle missing values
data['lag_7'].fillna(0, inplace=True)
data['rolling_mean_15'].fillna(data['rolling_mean_15'].mean(), inplace=True)

# Identify categorical and numerical columns 
categorical_cols = ['LOJA','day_of_week']  # Modify as necessary
numeric_cols = ['TOTAL_COLABORADORES', 'SKUS','PRODUTIVIDADE_HORA','SKUS_UP','SKUS_DOWN','ITEMS','SELF_CHECKOUT','lag_7', 'rolling_mean_15']

# Preprocessing for categorical data: OneHotEncoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
        ('num', 'passthrough', numeric_cols)  # Ensure numeric columns are passed through if not scaled
    ])

# Setup the pipeline 
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

# Define features and target
features = numeric_cols + categorical_cols  # Make sure this is updated if features change
target = 'VALOR_VENDA'

# Split the data
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

# Fit the model
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')

Mean Squared Error: 117322083.85925715
Mean Absolute Error: 3441.710358670849
R-squared: 0.9695355312699383


In [124]:
def forecast_and_compare(store_name, model, data, features):
    # Filter data for the specified store and sort by date
    store_data = data[data['LOJA'] == store_name].sort_values(by='DATA_VENDA')
    if store_data.empty:
        print("No data available for the specified store.")
        return {}, {}, {}
    
    # Find the last year of data available for the store
    last_year = store_data['DATA_VENDA'].dt.year.max() - 1
    historical_data = store_data[store_data['DATA_VENDA'].dt.year == last_year]

    # Calculate historical weekly averages
    weekly_averages = historical_data.groupby(historical_data['DATA_VENDA'].dt.day_name())['VALOR_VENDA'].mean().to_dict()

    # Prepare forecast for each day of the week in the next 7 days
    predictions = {}
    current_date = pd.Timestamp.today()  # Starting point for forecasting
    for i in range(7):
        forecast_date = current_date + pd.Timedelta(days=i)
        day_of_week = forecast_date.day_name()
        
        # Simulate the necessary input for the model ensuring the format matches training expectations
        input_data = store_data.iloc[-1][features].copy()  # Take the last available features
        input_data['DATA_VENDA'] = forecast_date  # Update the date for the forecast
        
        # Create a DataFrame from input data to match the model's expected input format
        formatted_input = pd.DataFrame([input_data])
        
        # Ensure categorical and date data is processed, if necessary
        # This line needs customization based on how 'day_of_week' is treated in your model training
        formatted_input['day_of_week'] = day_of_week
        
        # Predict using the model
        prediction = model.predict(formatted_input)[0]
        
        # Store prediction for each day
        predictions[day_of_week] = predictions.get(day_of_week, []) + [prediction]

    # Average predictions for each day if multiple predictions exist
    for day, values in predictions.items():
        predictions[day] = sum(values) / len(values)

    # Calculate differences from historical averages
    differences = {day: predictions[day] - weekly_averages.get(day, 0) for day in predictions}

    return predictions, weekly_averages, differences

# Example usage
store_name = input("Enter the store name: ")  # Get store name input
predictions, averages, differences = forecast_and_compare(store_name, model, data, features)
print("Predictions by day of the week:", predictions)
print("Historical averages by day of the week:", averages)
print("Differences from averages:", differences)

Enter the store name: Gaia
Predictions by day of the week: {'Friday': 108914.23404219572, 'Saturday': 109647.47661970889, 'Sunday': 108914.23404219572, 'Monday': 102796.64915854327, 'Tuesday': 108914.23404219572, 'Wednesday': 108914.23404219572, 'Thursday': 108914.23404219572}
Historical averages by day of the week: {'Friday': 105794.19788461538, 'Monday': 93089.48211538464, 'Saturday': 134454.23088050314, 'Sunday': 99980.61012820514, 'Thursday': 93888.86512820514, 'Tuesday': 88534.497948718, 'Wednesday': 90208.81576923077}
Differences from averages: {'Friday': 3120.03615758034, 'Saturday': -24806.754260794245, 'Sunday': 8933.623913990581, 'Monday': 9707.167043158624, 'Tuesday': 20379.736093477724, 'Wednesday': 18705.418272964947, 'Thursday': 15025.368913990576}


C:\Users\TeixeiJ\Anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  warnings.warn("Given feature/column names or counts do not match "
C:\Users\TeixeiJ\Anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  warnings.warn("Given feature/column names or counts do not match "
C:\Users\TeixeiJ\Anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  warnings.warn("Given feature/column names or counts do not match "
C:\Users\TeixeiJ\Anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning: Given feature/column names

In [125]:
# Before predicting, ensure that the input data frame is prepared correctly
input_data = store_data.iloc[-1][features].to_dict()
input_data['DATA_VENDA'] = forecast_date  # Simulate 'DATA_VENDA' if used in the model

# Convert to DataFrame
formatted_input = pd.DataFrame([input_data])
# Make sure that this DataFrame formatted_input has the same structure as the training data

NameError: name 'store_data' is not defined

In [126]:
# Example debug print statements
print("Feature names:", features)
print("X_train columns before fitting model:", X_train.columns.tolist())
print("Formatted input columns before prediction:", formatted_input.columns.tolist())

Feature names: ['TOTAL_COLABORADORES', 'SKUS', 'PRODUTIVIDADE_HORA', 'SKUS_UP', 'SKUS_DOWN', 'ITEMS', 'SELF_CHECKOUT', 'lag_7', 'rolling_mean_15', 'LOJA', 'day_of_week']
X_train columns before fitting model: ['TOTAL_COLABORADORES', 'SKUS', 'PRODUTIVIDADE_HORA', 'SKUS_UP', 'SKUS_DOWN', 'ITEMS', 'SELF_CHECKOUT', 'lag_7', 'rolling_mean_15', 'LOJA', 'day_of_week']


NameError: name 'formatted_input' is not defined